In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz # FuzzyWuzzy has a ratio function that calculates the standard Levenshtein distance similarity ratio between two sequences
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import os

## Stack datasets

In [2]:
de_data = pd.read_csv("LinkedIn_Data Engineer.csv")
de_data['Title Category'] = "Data Engineering"

In [3]:
ds_data = pd.read_csv("LinkedIn_Data Scientist.csv")
ds_data['Title Category'] = "Data Science"

In [4]:
da_data = pd.read_csv("LinkedIn_Analyst.csv")
da_data['Title Category'] = "Data Analytics"

In [184]:
all_data = pd.concat([de_data, ds_data, da_data], axis=0, ignore_index=True)

In [185]:
all_data = all_data.drop(columns=['index', 'Link', 'Level', 'Type', 'Applicants'])

In [186]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2908 entries, 0 to 2907
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              2908 non-null   object
 1   Date            2908 non-null   object
 2   Company         2908 non-null   object
 3   Title           2908 non-null   object
 4   Location        2908 non-null   object
 5   Description     2908 non-null   object
 6   Function        2908 non-null   object
 7   Industry        2908 non-null   object
 8   Title Category  2908 non-null   object
dtypes: object(9)
memory usage: 204.6+ KB


## Locations

In [165]:
pd.set_option('display.max_rows', 1000)

all_data.groupby(['Location']).count().sort_values(['ID'], ascending=False)['ID']

Location
United States                          110
New York, NY                           104
Washington, DC                          98
McLean, VA                              53
Austin, TX                              51
Houston, TX                             49
Chicago, IL                             48
Boston, MA                              47
Atlanta, GA                             47
Wayne, NJ                               41
Chantilly, VA                           38
Arlington, VA                           35
Dallas, TX                              35
Philadelphia, PA                        32
Reston, VA                              31
Cincinnati, OH                          30
San Francisco, CA                       29
Seattle, WA                             27
Tampa, FL                               26
Los Angeles, CA                         26
St Louis, MO                            25
Phoenix, AZ                             24
Pittsburgh, PA                          19
Mi

In [166]:
all_data[['location_part1', 'location_part2']] = all_data['Location'].str.split(",", 1, expand=True)

In [167]:
all_data[['Location', 'location_part1', 'location_part2']]

,Location,location_part1,location_part2
0,United States,United States,None
1,United States,United States,None
2,"Cincinnati, OH",Cincinnati,OH
3,"Beaverton, OR",Beaverton,OR
4,"Pasadena, CA",Pasadena,CA
...,...,...,...
2903,"California City, CA",California City,CA
2904,"Washington Highlands, MD",Washington Highlands,MD
2905,"Milwaukee, WI",Milwaukee,WI
2906,"West Baton Rouge Parish County, LA",West Baton Rouge Parish County,LA


In [168]:
all_data['location_part2'] = np.where(all_data['location_part2'] == " United States",
                                      all_data['location_part1'],
                                      all_data['location_part2'])

all_data['location_part2'] = all_data['location_part2'].str.strip()

In [169]:
all_data[(all_data['location_part2'].isnull()) & (all_data['location_part1'] != "United States")][['Location', 'location_part1', 'location_part2']]

,Location,location_part1,location_part2
177,Greater Phoenix Area,Greater Phoenix Area,None
180,Greater Cleveland,Greater Cleveland,None
271,Greater Hartford,Greater Hartford,None
486,New Bern-Morehead City Area,New Bern-Morehead City Area,None
542,Greater Houston,Greater Houston,None
728,Greater Philadelphia,Greater Philadelphia,None
829,Knoxville Metropolitan Area,Knoxville Metropolitan Area,None
833,Killeen-Temple Area,Killeen-Temple Area,None
910,Topeka Metropolitan Area,Topeka Metropolitan Area,None
914,Miami-Fort Lauderdale Area,Miami-Fort Lauderdale Area,None


In [170]:
# Replace locations that are in different format

location_dict = { "Greater Phoenix Area":        "AZ",
                        "Greater Cleveland":           "OH",
                        "Greater Hartford":            "CT",
                        "New Bern-Morehead City Area": "NC",
                        "Greater Houston":             "TX",
                        "Greater Philadelphia":        "PA",
                        "Knoxville Metropolitan Area": "TN",
                        "Killeen-Temple Area":         "TX",
                        "Topeka Metropolitan Area":    "KS",
                        "Miami-Fort Lauderdale Area":  "FL",
                        "Greater Dothan":              "NY",
                        "San Francisco Bay Area":      "CA",
                        "New York City Metropolitan Area": "NY",
                        "Greater Chicago Area":        "IL",
                        "Houma-Thibodaux Area":        "LA",
                        "Washington DC-Baltimore Area": "MD",
                        "Dallas-Fort Worth Metroplex":  "TX",
                        "San Diego Metropolitan Area":  "CA",
                        "Greater St. Louis":            "MO",
                        "Atlanta Metropolitan Area":    "GA",
                        "Omaha Metropolitan Area":      "IA",
                        "Salt Lake City Metropolitan Area": "UT",
                        "Buffalo-Niagara Falls Area":    "NY",
                        "Maine Metropolitan Area":       "ME",
                        "Texas Metropolitan Area":       "TX"
                      }

In [171]:
all_data['location_part3'] = all_data['Location'].map(location_dict)
all_data['location_part3'] = np.where(all_data['location_part3'].isnull(),
                                      all_data['location_part2'].map(location_dict),
                                      all_data['location_part3'])

In [173]:
all_data['location_part2'] = np.where( ((all_data['location_part2'].isnull()) | (all_data['location_part2'].str.contains("Metropolitan Area"))) & (all_data['location_part1'] != "United States"),
                                      all_data['location_part3'],
                                      all_data['location_part2'])

In [175]:
# Standardize states

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [176]:
all_data['State'] = all_data['location_part2'].map(us_state_to_abbrev)

In [177]:
all_data['State'] = np.where(all_data['State'].isnull(),
                             all_data['location_part2'],
                             all_data['State'])

In [178]:
all_data['State'] = all_data['State'].str.strip()

In [179]:
all_data[all_data['State'].isnull()]['location_part1'].unique()

array(['United States'], dtype=object)

In [180]:
CHARTS_FOLDER = '/Users/yinirong/UCLA_MSBA/Extracurricular/Data Science Job Market Project/Charts/'

In [182]:
# Output to Excel

# all_data[['ID','Company', 'Title Category', 'State']].to_excel(os.path.join(CHARTS_FOLDER, 'Locations.xlsx'), index=False)

## Industries

In [189]:
pd.set_option('display.max_rows', 1000)

industries = all_data.groupby(['Industry']).count().sort_values(['ID'], ascending=False)['ID'].to_frame()

In [191]:
# industries.to_excel(os.path.join(CHARTS_FOLDER, 'Industries.xlsx'))

In [193]:
industries = pd.read_excel(os.path.join(CHARTS_FOLDER, 'Industries.xlsx'))
industries.drop(columns = ['Count', 'Percent'], inplace=True)
industries.head()

,Industry,Industry_clean
0,Software Development,Software Development
1,IT Services and IT Consulting,IT Services and IT Consulting
2,Staffing and Recruiting,Staffing and Recruiting
3,Defense and Space Manufacturing,Manufacturing
4,"Technology, Information and Internet","Technology, Information and Internet"


In [194]:
industries_clean = all_data[['ID','Company', 'Title Category', 'Industry']].merge(industries,
                                                                                  how = 'left',
                                                                                  on = 'Industry')
industries_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2908 entries, 0 to 2907
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              2908 non-null   object
 1   Company         2908 non-null   object
 2   Title Category  2908 non-null   object
 3   Industry        2908 non-null   object
 4   Industry_clean  2908 non-null   object
dtypes: object(5)
memory usage: 136.3+ KB


In [197]:
# industries_clean.to_excel(os.path.join(CHARTS_FOLDER, 'Industries Clean.xlsx'), index=False)

## Remote work

In [204]:
len(all_data[all_data['Location'] == "United States"])

110

In [217]:
remote_work = all_data[(all_data['Location'] == "United States")][['Company', 'Title', 'Description']]
# remote_work.to_excel(os.path.join(CHARTS_FOLDER, 'Remote Locations.xlsx'), index=False)

In [218]:
remote_work['Title'] = remote_work['Title'].str.lower()

remote_work['is_remote1'] = remote_work['Title'].str.contains("remote")

remote_work.head()

,Company,Title,Description,is_remote1
0,Bloom Insurance,data engineer i,To support internal and external clients via p...,False
1,Intrepid,data engineer(remote),Intrepid sets the standard for delivering exce...,True
5,BairesDev,etl + data engineer - remote work,Who We are BairesDev is proud to be the faste...,True
27,"SMX Services & Consulting, Inc.",data engineer,Job Description SMX Services & Consulting ...,False
31,BairesDev,data science / data engineer - remote work,Who We are BairesDev is proud to be the faste...,True


In [219]:
len(remote_work[remote_work['is_remote1'] == True])

31

In [220]:
remote_work['Description'] = remote_work['Description'].str.lower()

remote_work['is_remote2'] = remote_work['Description'].str.contains("remote")

remote_work.head()

,Company,Title,Description,is_remote1,is_remote2
0,Bloom Insurance,data engineer i,to support internal and external clients via p...,False,False
1,Intrepid,data engineer(remote),intrepid sets the standard for delivering exce...,True,True
5,BairesDev,etl + data engineer - remote work,who we are bairesdev is proud to be the faste...,True,True
27,"SMX Services & Consulting, Inc.",data engineer,job description smx services & consulting ...,False,False
31,BairesDev,data science / data engineer - remote work,who we are bairesdev is proud to be the faste...,True,True


In [221]:
len(remote_work[remote_work['is_remote2'] == True])

66

In [222]:
remote_work['is_remote'] = np.where((remote_work['is_remote1']==True) | (remote_work['is_remote2']==True), 1, 0)

In [223]:
remote_work.head()

,Company,Title,Description,is_remote1,is_remote2,is_remote
0,Bloom Insurance,data engineer i,to support internal and external clients via p...,False,False,0
1,Intrepid,data engineer(remote),intrepid sets the standard for delivering exce...,True,True,1
5,BairesDev,etl + data engineer - remote work,who we are bairesdev is proud to be the faste...,True,True,1
27,"SMX Services & Consulting, Inc.",data engineer,job description smx services & consulting ...,False,False,0
31,BairesDev,data science / data engineer - remote work,who we are bairesdev is proud to be the faste...,True,True,1


In [224]:
len(remote_work[remote_work['is_remote'] == 1])

72